In [1]:
import pandas as pd
from scipy.stats import t
import numpy as np
import math
import random
import time
from dateutil import parser
import datetime as dtime
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 25)

In [2]:
start_time = time.time()
today = dtime.datetime(2022, 6, 1)

In [3]:
#df = pd.read_csv("https://projects.fivethirtyeight.com/polls-page/data/generic_ballot_polls.csv", parse_dates=['start_date','end_date', 'created_at'])
df = pd.read_csv("C:/Users/horat/Python Project/Senate 2022/Test Forecast/generic_ballot_polls_historical.csv", parse_dates=['start_date','end_date', 'created_at'])
psters = pd.read_csv("C:/Users/horat/Python Project/Senate 2022/Test Forecast/pollster-ratings.csv")
output = pd.read_csv("C:/Users/horat/Python Project/Senate 2022/Test Forecast/generic-ballot.csv")

In [4]:
df=df[(df['end_date'] < '2022-06-01') & (df['end_date']>'2021-04-01')]

In [5]:
# https://fivethirtyeight.com/features/how-the-fivethirtyeight-senate-forecast-model-works/
def assign_weight(n, days, g, v, internal, dfe): # n=sample size, days=days_old, v=grade, internal=pop_value
    # impute missing sample size
    if n=="":
        n=df['sample_size'].median # or 400 for district-level polls, 600 for state-level polls and 1,200 for national polls
    else:
        pass
    # weight by sample size
    if n > 3500:
        n_weight = 55.22
    else:
        n_weight = 5 / (1 / math.pow(n, .3)) # https://fivethirtyeight.com/features/polls-now-weighted-by-sample-size/
    # weight by recency
    aggr = 8 * math.log(dfe / 20 + 1) + 30
    recency = (90 / (1 + math.pow(1.5, (days - aggr) / (aggr / 10))) + 10) / 100
    # internal polls weight
    if internal == "":
        partisan_effect = 1
    else: 
        partisan_effect = 0.7
    w = n_weight * v * partisan_effect * recency * g
    return w

In [6]:
# pollster df to dictionary -> id: letter_grade, grade_value, house_bias
def pollster_id_grade_bias(df):
    return dict([(rating_id,[grade,grarde_value,house_bias ]) for rating_id, grade,grarde_value,house_bias in zip(df.pollster_rating_id, df.grade,df.grade_value,df.house_bias)])

In [7]:
def add_pollsters_info(df):
    pop_val = {'v' : 1, 'lv' : 1, 'rv' : 0.67, 'a' : 0.5, '' : 0.5} # create voter grades
    df['pop_value'] = df['population'].map(pop_val)
    for index, row in df.iterrows():
        try: # catch pollsters which don't exist in pollster_rating information
            df.at[index, "grade_symbol"]=pollster_id_grade_bias(psters)[row['pollster_rating_id']][0]
            df.at[index, "grade"]=pollster_id_grade_bias(psters)[row['pollster_rating_id']][1]
            df.at[index, "house_bias"]=pollster_id_grade_bias(psters)[row['pollster_rating_id']][2]
        except:
            df.at[index, "house_bias"] = 0
            df.at[index, "grade"] = 0.6
            df.at[index, "grade_symbol"] = ""
    return df

In [8]:
df=add_pollsters_info(df)
df

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,fte_grade,methodology,state,start_date,end_date,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,question_id,sample_size,population,subpopulation,population_full,tracking,created_at,notes,url,source,internal,partisan,race_id,cycle,office_type,seat_number,seat_name,election_date,stage,nationwide_batch,dem,rep,ind,pop_value,grade_symbol,grade,house_bias
509,80064,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B,Online,NaN,2022-05-29,2022-05-31,NaN,NaN,NaN,160499,13228.0,rv,NaN,rv,True,2022-09-08 10:52:00,NaN,https://morningconsul...,NaN,False,NaN,8990,2022,U.S. House,NaN,Generic,11/8/22,general,False,44.0,42.0,NaN,0.67,B,1.00,0.2757
510,79055,568,YouGov,352,Economist,YouGov,391,YouGov,B+,Online,NaN,2022-05-28,2022-05-31,NaN,NaN,NaN,156020,776.0,rv,NaN,rv,NaN,2022-06-01 09:55:00,NaN,https://docs.cdn.youg...,NaN,False,NaN,8990,2022,U.S. House,NaN,Generic,11/8/22,general,False,43.0,43.0,NaN,0.67,B-,0.93,0.3595
511,80065,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B,Online,NaN,2022-05-28,2022-05-30,NaN,NaN,NaN,160500,13249.0,rv,NaN,rv,True,2022-09-08 10:52:00,NaN,https://morningconsul...,NaN,False,NaN,8990,2022,U.S. House,NaN,Generic,11/8/22,general,False,43.0,42.0,NaN,0.67,B,1.00,0.2757
512,80066,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B,Online,NaN,2022-05-27,2022-05-29,NaN,NaN,NaN,160501,12522.0,rv,NaN,rv,True,2022-09-08 10:52:00,NaN,https://morningconsul...,NaN,False,NaN,8990,2022,U.S. House,NaN,Generic,11/8/22,general,False,43.0,43.0,NaN,0.67,B,1.00,0.2757
513,80067,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B,Online,NaN,2022-05-26,2022-05-28,NaN,NaN,NaN,160502,12058.0,rv,NaN,rv,True,2022-09-08 10:52:00,NaN,https://morningconsul...,NaN,False,NaN,8990,2022,U.S. House,NaN,Generic,11/8/22,general,False,43.0,42.0,NaN,0.67,B,1.00,0.2757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1273,80183,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B,Online,NaN,2021-04-04,2021-04-06,NaN,NaN,NaN,160618,13727.0,rv,NaN,rv,True,2022-09-08 12:34:00,NaN,https://morningconsul...,NaN,False,NaN,8990,2022,U.S. House,NaN,Generic,11/8/22,general,False,44.0,41.0,NaN,0.67,B,1.00,0.2757
1274,80182,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B,Online,NaN,2021-04-03,2021-04-05,NaN,NaN,NaN,160617,13900.0,rv,NaN,rv,True,2022-09-08 12:34:00,NaN,https://morningconsul...,NaN,False,NaN,8990,2022,U.S. House,NaN,Generic,11/8/22,general,False,44.0,41.0,NaN,0.67,B,1.00,0.2757
1275,80181,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B,Online,NaN,2021-04-02,2021-04-04,NaN,NaN,NaN,160616,13597.0,rv,NaN,rv,True,2022-09-08 12:34:00,NaN,https://morningconsul...,NaN,False,NaN,8990,2022,U.S. House,NaN,Generic,11/8/22,general,False,44.0,41.0,NaN,0.67,B,1.00,0.2757
1276,80180,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B,Online,NaN,2021-04-01,2021-04-03,NaN,NaN,NaN,160615,12400.0,rv,NaN,rv,True,2022-09-08 12:34:00,NaN,https://morningconsul...,NaN,False,NaN,8990,2022,U.S. House,NaN,Generic,11/8/22,general,False,44.0,41.0,NaN,0.67,B,1.00,0.2757


In [9]:
# there are polls with the same poll_id overlap, only the more reliable sample type is chosen via pop_value
fp = pd.DataFrame()
fp = df.groupby("poll_id")["pop_value"].max().reset_index().merge(df, on=["poll_id", "pop_value"], how="inner")
# organize df1 with date
fp.sort_values(by='end_date', ascending = False, inplace = True)
fp=fp.reset_index(drop=True)
# reorganize columns order to that of df
column_headers = list(df.columns.values)
fp=fp.loc[:,column_headers]
fp

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,fte_grade,methodology,state,start_date,end_date,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,question_id,sample_size,population,subpopulation,population_full,tracking,created_at,notes,url,source,internal,partisan,race_id,cycle,office_type,seat_number,seat_name,election_date,stage,nationwide_batch,dem,rep,ind,pop_value,grade_symbol,grade,house_bias
0,79055,568,YouGov,352,Economist,YouGov,391,YouGov,B+,Online,NaN,2022-05-28,2022-05-31,NaN,NaN,NaN,156020,776.0,rv,NaN,rv,NaN,2022-06-01 09:55:00,NaN,https://docs.cdn.youg...,NaN,False,NaN,8990,2022,U.S. House,NaN,Generic,11/8/22,general,False,43.0,43.0,NaN,0.67,B-,0.93,0.3595
1,80064,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B,Online,NaN,2022-05-29,2022-05-31,NaN,NaN,NaN,160499,13228.0,rv,NaN,rv,True,2022-09-08 10:52:00,NaN,https://morningconsul...,NaN,False,NaN,8990,2022,U.S. House,NaN,Generic,11/8/22,general,False,44.0,42.0,NaN,0.67,B,1.00,0.2757
2,80065,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B,Online,NaN,2022-05-28,2022-05-30,NaN,NaN,NaN,160500,13249.0,rv,NaN,rv,True,2022-09-08 10:52:00,NaN,https://morningconsul...,NaN,False,NaN,8990,2022,U.S. House,NaN,Generic,11/8/22,general,False,43.0,42.0,NaN,0.67,B,1.00,0.2757
3,80066,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B,Online,NaN,2022-05-27,2022-05-29,NaN,NaN,NaN,160501,12522.0,rv,NaN,rv,True,2022-09-08 10:52:00,NaN,https://morningconsul...,NaN,False,NaN,8990,2022,U.S. House,NaN,Generic,11/8/22,general,False,43.0,43.0,NaN,0.67,B,1.00,0.2757
4,80067,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B,Online,NaN,2022-05-26,2022-05-28,NaN,NaN,NaN,160502,12058.0,rv,NaN,rv,True,2022-09-08 10:52:00,NaN,https://morningconsul...,NaN,False,NaN,8990,2022,U.S. House,NaN,Generic,11/8/22,general,False,43.0,42.0,NaN,0.67,B,1.00,0.2757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,80183,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B,Online,NaN,2021-04-04,2021-04-06,NaN,NaN,NaN,160618,13727.0,rv,NaN,rv,True,2022-09-08 12:34:00,NaN,https://morningconsul...,NaN,False,NaN,8990,2022,U.S. House,NaN,Generic,11/8/22,general,False,44.0,41.0,NaN,0.67,B,1.00,0.2757
719,80182,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B,Online,NaN,2021-04-03,2021-04-05,NaN,NaN,NaN,160617,13900.0,rv,NaN,rv,True,2022-09-08 12:34:00,NaN,https://morningconsul...,NaN,False,NaN,8990,2022,U.S. House,NaN,Generic,11/8/22,general,False,44.0,41.0,NaN,0.67,B,1.00,0.2757
720,80181,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B,Online,NaN,2021-04-02,2021-04-04,NaN,NaN,NaN,160616,13597.0,rv,NaN,rv,True,2022-09-08 12:34:00,NaN,https://morningconsul...,NaN,False,NaN,8990,2022,U.S. House,NaN,Generic,11/8/22,general,False,44.0,41.0,NaN,0.67,B,1.00,0.2757
721,80180,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B,Online,NaN,2021-04-01,2021-04-03,NaN,NaN,NaN,160615,12400.0,rv,NaN,rv,True,2022-09-08 12:34:00,NaN,https://morningconsul...,NaN,False,NaN,8990,2022,U.S. House,NaN,Generic,11/8/22,general,False,44.0,41.0,NaN,0.67,B,1.00,0.2757


In [10]:
max_output = parser.parse(max(output["date"])) # the latest date in the file
max_output

datetime.datetime(2022, 5, 17, 0, 0)

In [11]:
if (today - max_output).days == 1:
    start_date=today
else:
    start_date=max_output
num_days = (today - start_date).days
num_days

15

In [12]:
output = output[output["date"] < start_date.strftime("%Y-%m-%d")]
output

,date,rep,dem,margin,weight
0,2021-04-01,40.47,45.00,-4.54,78.78
1,2021-04-02,40.46,45.01,-4.55,76.79
2,2021-04-03,40.45,45.02,-4.57,74.75
3,2021-04-04,40.45,45.03,-4.58,72.66
4,2021-04-05,40.44,45.04,-4.60,70.54
...,...,...,...,...,...
406,2022-05-12,45.55,43.64,1.91,470.15
407,2022-05-13,45.47,43.64,1.83,501.10
408,2022-05-14,45.48,43.65,1.82,495.25
409,2022-05-15,45.48,43.66,1.82,489.23


In [13]:
pavg = pd.DataFrame()

In [14]:
for z in range(num_days + 1):
    sim_date = start_date + dtime.timedelta(days=z)
    # FILTER TO NEWEST POLL #
    op = fp[fp["end_date"] < sim_date].copy()
    u_ids = op["pollster_id"].unique()
    op['days_old'] = (sim_date - op['end_date']).dt.days
    pfsp = pd.DataFrame()
    for e in u_ids:
        psp = op[op["pollster_id"] == e]
        newest = max(psp["end_date"])
        dsp = psp[psp["end_date"] == newest]
        pfsp = pd.concat([pfsp, dsp], ignore_index=True)
    for index, row in pfsp.iterrows():
        weight = assign_weight(row["sample_size"], row["days_old"],
                               row["grade"], row["pop_value"], row["partisan"], 60)
        pfsp.at[index, "weight"] = round(weight, 3)
        partisan_effect = -3 if row["partisan"] == "REP" else 3 if row["partisan"] == "DEM" else 0
        house_bias = row["house_bias"]
        bias = (house_bias + partisan_effect)
        row["rep"] = row["rep"] + (bias / 2)
        row["dem"] = row["dem"] - (bias / 2)
        pfsp.at[index, "rep_weight"] = round(weight * row["rep"], 3)
        pfsp.at[index, "dem_weight"] = round(weight * row["dem"], 3)
    avgs = pd.DataFrame({'date': [sim_date.strftime("%Y-%m-%d")],
                         'rep': [round(pfsp["rep_weight"].sum() / pfsp["weight"].sum(), 2)],
                         'dem': [round(pfsp["dem_weight"].sum() / pfsp["weight"].sum(), 2)],
                         'margin': [
                             round((pfsp["rep_weight"].sum() - pfsp["dem_weight"].sum()) / pfsp["weight"].sum(), 2)],
                         'weight': round(pfsp["weight"].sum(), 2)})
    print(sim_date, ":", avgs["margin"][0])
    output = pd.concat([output, avgs], ignore_index=True)
    print(output)

2022-05-17 00:00:00 : 1.31
           date    rep    dem  margin  weight
0    2021-04-01  40.47  45.00   -4.54   78.78
1    2021-04-02  40.46  45.01   -4.55   76.79
2    2021-04-03  40.45  45.02   -4.57   74.75
3    2021-04-04  40.45  45.03   -4.58   72.66
4    2021-04-05  40.44  45.04   -4.60   70.54
..          ...    ...    ...     ...     ...
407  2022-05-13  45.47  43.64    1.83  501.10
408  2022-05-14  45.48  43.65    1.82  495.25
409  2022-05-15  45.48  43.66    1.82  489.23
410  2022-05-16  45.51  43.74    1.77  497.31
411  2022-05-17  44.98  43.67    1.31  573.79

[412 rows x 5 columns]
2022-05-18 00:00:00 : 1.4
           date    rep    dem  margin  weight
0    2021-04-01  40.47  45.00   -4.54   78.78
1    2021-04-02  40.46  45.01   -4.55   76.79
2    2021-04-03  40.45  45.02   -4.57   74.75
3    2021-04-04  40.45  45.03   -4.58   72.66
4    2021-04-05  40.44  45.04   -4.60   70.54
..          ...    ...    ...     ...     ...
408  2022-05-14  45.48  43.65    1.82  495.25
409

In [15]:
output.to_csv('C:/Users/horat/Python Project/Senate 2022/Test Forecast/generic-ballot.csv', index=False, header=True)